In [1]:
import pandas as pd
import time
import numpy as np
np.seterr(invalid='ignore')
import nbimporter
from PPP import PPP

prevGames = ['Game1','Game2', 'Game3']
Opp = 'Nuggets'
Team = 'Heat'
series = 'Finals'

In [2]:
players = ['adebayo', 'butler', 'highsmith', 'love', 'lowry', 'martin', 'robinson', 'strus', 'vincent', 'zeller', 'team', 'opp']

In [32]:
def load_data(players):
    
    # Establish dictionaries for new game data and overall series/season data
    new_player_data = {}
    ovr_player_data = {}
    ovr_series_player_data = {}
    team_opp = ['team', 'opp']
    
    for game in prevGames: # Loop over all games in series/season
        
        for player in players: # Loop over all players on roster
            
            if player == 'team': # Add all player data together to create team file for game

                try: # to load a team file for the game (helpful if already added team file, and/or rerunning for Opp file)
                    newplayerFile = (f'Team/{game + Opp}Team.csv')
                    newplayer = pd.read_csv(newplayerFile)
                    newplayer = newplayer.dropna()
                    newplayer = newplayer.set_index('Shot Type')
                    newplayer = newplayer.astype('int')
                    new_player_data[player] = newplayer # Add the existing new team file to the new_player_data dictionary
                except(FileNotFoundError):
                    # Load an empty df
                    newplayerFile = ('Team/empty.csv')
                    newplayer = pd.read_csv(newplayerFile)
                    newplayer = newplayer.set_index('Shot Type')

                    for key, df in new_player_data.items(): # Loop over the new_player_data and add together all player dataframes
                        if key not in team_opp: # Exclude the team and opponent dataframes
                            newplayer = newplayer.add(df)

                    new_player_data['team'] = newplayer # Add the created new team file to new_player_data dictionary
                    newplayer.to_csv(f'../{Opp}/Opp/{game + Team}Opp.csv') # Save team file to opponent folder for their opponent stats
                    newplayer.to_csv(f'Team/{game + Opp}Team.csv') # Save team file to team folder
                    
                # Load overall data for the team
                try:
                    ovrplayerFile = (f'!Data/Team.csv')
                    ovrplayer = pd.read_csv(ovrplayerFile)
                    ovrplayer = ovrplayer.dropna()
                    ovrplayer = ovrplayer.set_index('Shot Type')
                    ovrplayer = ovrplayer.astype('int')
                    ovr_player_data[player] = ovrplayer # Add the overall team data to the ovr_player_data dictionary
                except(FileNotFoundError):
                    ovrplayer = newplayer
                    ovr_player_data[player] = ovrplayer
                    ovrplayer.to_csv(f'!Data/{player.title()}.csv')
                
                # Load overall series data for the player
                try: 
                    ovr_seriesplayerFile = (f'!Data/{series}/{player.title() + series}.csv')
                    ovr_seriesplayer = pd.read_csv(ovr_seriesplayerFile)
                    ovr_seriesplayer = ovr_seriesplayer.dropna()
                    ovr_seriesplayer = ovr_seriesplayer.set_index('Shot Type')
                    ovr_seriesplayer = ovr_seriesplayer.astype('int')
                    ovr_series_player_data[player] = ovr_seriesplayer # Add overall player data to the ovr_player_data dictionary
                # If there is no overall series data for the player, set the new player data as the overall series data and continue to the next player    
                except(FileNotFoundError):
                    ovr_seriesplayer = newplayer
                    ovr_series_player_data[player] = ovr_seriesplayer
                    ovr_seriesplayer.to_csv(f'!Data/{series}/{player.title() + series}.csv')
                    continue
                    
                if newplayer.columns.equals(ovrplayer.columns) and newplayer.columns.equals(ovr_seriesplayer.columns): # Make sure the new and overall dataframes can be added without error
                    ovrplayer = ovrplayer.add(newplayer)
                    ovr_player_data[player] = ovrplayer
                    ovrplayer.to_csv(f'!Data/{player.title()}.csv') # Write the updated overall player file to the overall data directory
                    if game == 'Game1':
                        continue

                    ovr_seriesplayer = ovr_seriesplayer.add(newplayer)
                    ovr_series_player_data[player] = ovr_seriesplayer
                    ovr_seriesplayer.to_csv(f'!Data/{series}/{player.title() + series}.csv' )# Write the updated overall series player file to the overall series data directory
                else:
                    print(f"{player.title()}'s columns do not match")  
                continue

            if player == 'opp': # Load the opponent file and add the new game data to the overall opponent file if necessary

                try: # to load the new opponent file (if running this teams data first, there will be no opponent file!!)
                    newplayerFile = (f'Opp/{game + Opp}Opp.csv')
                    newplayer = pd.read_csv(newplayerFile)
                    newplayer = newplayer.dropna()
                    newplayer = newplayer.set_index('Shot Type')
                    newplayer = newplayer.astype('int')
                    new_player_data[player] = newplayer # Add the new opponent file to the new_player_data dictionary
                except(FileNotFoundError):
                    print(f'No {player.title()} file found.')
                    continue
                    
                # Load overall data for the opponent
                try:
                    ovrplayerFile = (f'!Data/Opp.csv')
                    ovrplayer = pd.read_csv(ovrplayerFile)
                    ovrplayer = ovrplayer.dropna()
                    ovrplayer = ovrplayer.set_index('Shot Type')
                    ovrplayer = ovrplayer.astype('int')
                    ovr_player_data[player] = ovrplayer # Add the overall opponent file to the ovr_player_data dictionary
                except(FileNotFoundError):
                    if newplayer:
                        ovrplayer = newplayer
                        ovr_player_data[player] = ovrplayer
                        ovrplayer.to_csv(f'!Data/{player.title()}.csv')
                    else:
                        print(f'No overall Opponent file found.')
                        continue
                        
                # Load overall series data for the player
                try: 
                    ovr_seriesplayerFile = (f'!Data/{series}/{player.title() + series}.csv')
                    ovr_seriesplayer = pd.read_csv(ovr_seriesplayerFile)
                    ovr_seriesplayer = ovr_seriesplayer.dropna()
                    ovr_seriesplayer = ovr_seriesplayer.set_index('Shot Type')
                    ovr_seriesplayer = ovr_seriesplayer.astype('int')
                    ovr_series_player_data[player] = ovr_seriesplayer # Add overall player data to the ovr_player_data dictionary
                # If there is no overall series data for the player, set the new player data as the overall series data and continue to the next player    
                except(FileNotFoundError):
                    ovr_seriesplayer = newplayer
                    ovr_series_player_data[player] = ovr_seriesplayer
                    ovr_seriesplayer.to_csv(f'!Data/{series}/{player.title() + series}.csv')
                    continue

                if newplayer.columns.equals(ovrplayer.columns) and newplayer.columns.equals(ovr_seriesplayer.columns): # Make sure the new and overall dataframes can be added without error
                    ovrplayer = ovrplayer.add(newplayer)
                    ovr_player_data[player] = ovrplayer
                    ovrplayer.to_csv(f'!Data/{player.title()}.csv') # Write the updated overall player file to the overall data directory
                    if game == 'Game1':
                        continue

                    ovr_seriesplayer = ovr_seriesplayer.add(newplayer)
                    ovr_series_player_data[player] = ovr_seriesplayer
                    ovr_seriesplayer.to_csv(f'!Data/{series}/{player.title() + series}.csv' )# Write the updated overall series player file to the overall series data directory
                else:
                    print(f"{player.title()}'s columns do not match")  
                continue
                
            ### Loading/Adding the player files (first part of the loop)###
            try:
                newplayerFile = (f'{player.title()}/{game + Opp + player.title()}.csv') # Search the team directory for the player, then concat the game/opp/player to find csv for game
                newplayer = pd.read_csv(newplayerFile)
                newplayer = newplayer.dropna()
                newplayer = newplayer.set_index('Shot Type')
                newplayer = newplayer.astype('int')
                new_player_data[player] = newplayer # Add the new player data to the new_player_data dictionary
            except(FileNotFoundError):
                print(f'No {player.title()} file found for {game}.')
                try: # to load an overall file for the player
                    ovrplayerFile = (f'!Data/{player.title()}.csv')
                    ovrplayer = pd.read_csv(ovrplayerFile)
                    ovrplayer = ovrplayer.dropna()
                    ovrplayer = ovrplayer.set_index('Shot Type')
                    ovrplayer = ovrplayer.astype('int')
                    ovr_player_data[player] = ovrplayer # Add overall player data to the ovr_player_data dictionary
                    continue
                except(FileNotFoundError):
                    print(f'No data for {player.title()}.')
                    continue # If no player data is found, skip and go to the next player (edit for later)
                    
                try: # to load an overall player file for the series
                    ovr_seriesplayerFile = (f'!Data/{player.title()}.csv')
                    ovr_seriesplayer = pd.read_csv(ovr_seriesplayerFile)
                    ovr_seriesplayer = ovr_seriesplayer.dropna()
                    ovr_seriesplayer = ovr_seriesplayer.set_index('Shot Type')
                    ovr_seriesplayer = ovr_seriesplayer.astype('int')
                    ovr_series_player_data[player] = ovr_seriesplayer # Add overall player data to the ovr_series_player_data dictionary
                    continue
                except(FileNotFoundError):
                    print(f'No {series} series data for {player.title()}.')
                    continue # If no player data is found, skip and go to the next player (edit for later)
                    
            # Load overall data for the player
            try: 
                ovrplayerFile = (f'!Data/{player.title()}.csv')
                ovrplayer = pd.read_csv(ovrplayerFile)
                ovrplayer = ovrplayer.dropna()
                ovrplayer = ovrplayer.set_index('Shot Type')
                ovrplayer = ovrplayer.astype('int')
                ovr_player_data[player] = ovrplayer # Add overall player data to the ovr_player_data dictionary
            # If there is no overall data for the player, set the new player data as the overall and continue to the next player    
            except(FileNotFoundError):
                ovrplayer = newplayer
                ovr_player_data[player] = ovrplayer
                ovrplayer.to_csv(f'!Data/{player.title()}.csv')
              
            # Load overall series data for the player
            try: 
                ovr_seriesplayerFile = (f'!Data/{series}/{player.title() + series}.csv')
                ovr_seriesplayer = pd.read_csv(ovr_seriesplayerFile)
                ovr_seriesplayer = ovr_seriesplayer.dropna()
                ovr_seriesplayer = ovr_seriesplayer.set_index('Shot Type')
                ovr_seriesplayer = ovr_seriesplayer.astype('int')
                ovr_series_player_data[player] = ovr_seriesplayer # Add overall player data to the ovr_player_data dictionary
            # If there is no overall series data for the player, set the new player data as the overall series data and continue to the next player    
            except(FileNotFoundError):
                ovr_seriesplayer = newplayer
                ovr_series_player_data[player] = ovr_seriesplayer
                ovr_seriesplayer.to_csv(f'!Data/{series}/{player.title() + series}.csv')
                continue
                
            if newplayer.columns.equals(ovrplayer.columns) and newplayer.columns.equals(ovr_seriesplayer.columns): # Make sure the new and overall dataframes can be added without error
                ovrplayer = ovrplayer.add(newplayer)
                ovr_player_data[player] = ovrplayer
                ovrplayer.to_csv(f'!Data/{player.title()}.csv') # Write the updated overall player file to the overall data directory
                if game == 'Game1':
                    continue
                    
                ovr_seriesplayer = ovr_seriesplayer.add(newplayer)
                ovr_series_player_data[player] = ovr_seriesplayer
                ovr_seriesplayer.to_csv(f'!Data/{series}/{player.title() + series}.csv' )# Write the updated overall series player file to the overall series data directory
            else:
                print(f"{player.title()}'s columns do not match")  
            continue
            
    return new_player_data, ovr_player_data, ovr_series_player_data # Return both dictionaries as a tuple

new_player_data, ovr_player_data, ovr_series_player_data  = load_data(players) # Unpack the tuple of dictionaries into their respective names

No Love file found for Game1.
No Highsmith file found for Game3.


In [4]:
newButlerStats = PPP(new_player_data['butler'])
newButlerStats.to_csv(f'Butler/PPP Stats/{prevGames[-1] + Opp}.csv')
newButlerStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.79,35.9,1,38.5,0.78,34.6,33.3,7,28.6,2,50.0,0.8,38.5,50.0,3,66.7,1,0.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,1.14,2.3,0,N/A,1.14,3.4,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,2.0,5.1,0,100.0,2.0,7.7,100.0,2,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.87,38.2,0,35.7,1.02,30.3,42.9,6,50.0,1,0.0,0.71,53.8,28.6,3,33.3,4,25.0
TRANSITION,1.0,5.1,0,50.0,2.0,3.8,100.0,1,100.0,0,N/A,0.0,7.7,0.0,1,0.0,0,N/A
Attacking Closeouts,0.69,7.4,1,0.0,0.69,11.1,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,0.0,2.6,0,0.0,0.0,3.8,0.0,0,N/A,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A


In [5]:
newButlerSeriesStats = PPP(ovr_series_player_data['butler'])
newButlerSeriesStats.to_csv(f'Butler/PPP Stats/{prevGames[-1] + Opp}.csv')
newButlerSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.04,38.7,4,41.4,0.41,34.3,20.6,28,21.4,6,16.7,1.88,46.8,70.8,10,50.0,14,85.7
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.0,1.8,0,0.0,0.0,2.8,0.0,3,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,1.14,0.5,0,N/A,1.14,0.8,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,1.33,1.8,0,66.7,2.0,1.9,100.0,2,100.0,0,N/A,0.0,1.7,0.0,0,N/A,1,0.0
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.38,32.4,0,56.2,1.41,27.0,56.5,18,66.7,5,20.0,1.36,42.4,56.0,10,80.0,15,40.0
TRANSITION,0.57,4.2,0,28.6,0.67,5.6,33.3,6,33.3,0,N/A,0.0,1.7,0.0,1,0.0,0,N/A
Attacking Closeouts,0.98,6.7,2,42.9,0.98,10.4,42.9,4,75.0,3,0.0,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,2.57,4.2,0,85.7,2.57,6.5,85.7,0,N/A,7,85.7,N/A,0.0,N/A,0,N/A,0,N/A


In [6]:
butlerStats = PPP(ovr_player_data['butler'])
butlerStats.to_csv('!PPP Stats/butlerStats.csv')
butlerStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.87,32.2,10,36.1,0.62,30.8,26.9,79,29.1,14,14.3,1.28,34.9,52.9,23,43.5,28,60.7
PNR Screener,0.0,0.4,0,0.0,0.0,0.3,0.0,1,0.0,0,N/A,0.0,0.6,0.0,0,N/A,1,0.0
DHO Ball Handler,0.6,1.0,0,20.0,0.6,1.5,20.0,4,0.0,1,100.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,0.76,1.6,1,33.3,1.14,0.3,N/A,0,N/A,0,N/A,0.71,4.1,33.3,2,50.0,4,25.0
DBL Ball Handler,0.5,1.6,0,25.0,1.33,0.9,66.7,2,100.0,1,0.0,0.0,2.9,0.0,0,N/A,5,0.0
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.2,39.6,7,49.1,1.04,36.5,42.0,79,43.0,21,38.1,1.47,45.6,58.7,23,78.3,52,50.0
TRANSITION,1.26,6.0,2,64.0,1.21,7.0,60.0,20,60.0,0,N/A,1.45,4.0,80.0,3,66.7,2,100.0
Attacking Closeouts,0.74,4.0,7,45.5,0.74,6.1,45.5,8,62.5,3,0.0,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,2.64,2.2,0,90.9,2.64,3.3,90.9,1,100.0,10,90.0,N/A,0.0,N/A,0,N/A,0,N/A


In [7]:
newAdebayoStats = PPP(new_player_data['adebayo'])
newAdebayoStats.to_csv(f'Adebayo/PPP Stats/{prevGames[-1] + Opp}.csv')
newAdebayoStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,0.87,25.5,0,33.3,0.87,29.9,33.3,6,33.3,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,1.33,22.2,0,50.0,1.0,8.7,50.0,2,50.0,0,N/A,1.5,100.0,50.0,0,N/A,4,50.0
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.04,10.7,0,50.0,1.04,12.5,50.0,2,50.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,1.37,13.5,0,0.0,1.37,15.8,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,0.0,3.7,0,0.0,0.0,4.3,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [8]:
newAdebayoSeriesStats = PPP(ovr_series_player_data['adebayo'])
newAdebayoSeriesStats.to_csv(f'Adebayo/PPP Stats/{prevGames[-1] + Opp}.csv')
newAdebayoSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.0,2.4,0,0.0,0.0,3.1,0.0,3,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,1.24,25.3,5,61.9,1.25,25.9,55.6,18,55.6,0,N/A,1.23,23.6,100.0,3,100.0,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,1.13,21.0,3,52.4,1.17,12.5,58.3,12,58.3,0,N/A,1.09,47.2,44.4,2,100.0,7,28.6
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.81,28.3,1,41.2,0.87,32.2,43.3,30,43.3,0,N/A,0.4,16.1,25.0,1,100.0,3,0.0
TRANSITION,1.57,6.5,0,33.3,1.75,6.5,0.0,1,0.0,0,N/A,1.0,6.5,50.0,1,100.0,1,0.0
Attacking Closeouts,1.0,1.6,0,50.0,1.0,2.1,50.0,2,50.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,2.0,0.8,0,100.0,2.0,1.0,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [9]:
adebayoStats = PPP(ovr_player_data['adebayo'])
adebayoStats.to_csv('!PPP Stats/adebayoStats.csv')
adebayoStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.92,1.9,0,20.0,0.92,2.6,20.0,5,20.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,1.09,19.6,22,68.6,1.06,22.2,64.4,45,64.4,0,N/A,1.26,12.7,100.0,3,100.0,3,100.0
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,0.93,19.5,4,40.0,0.55,11.1,26.7,30,26.7,0,N/A,1.2,41.2,50.0,16,75.0,24,33.3
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.01,38.2,12,52.2,0.97,42.3,48.7,113,48.7,0,N/A,1.16,27.4,68.0,18,83.3,7,28.6
TRANSITION,1.41,7.2,1,59.1,1.55,4.2,66.7,6,66.7,0,N/A,1.3,14.9,56.2,7,100.0,9,22.2
Attacking Closeouts,1.43,1.7,0,71.4,1.43,2.4,71.4,7,71.4,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,2.0,1.2,0,100.0,2.0,1.7,100.0,5,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [10]:
newLoveStats = PPP(new_player_data['love'])
newLoveStats.to_csv(f'Love/PPP Stats/{prevGames[-1] + Opp}.csv')
newLoveStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DHO Ball Handler,0.0,20.0,0,0.0,0.0,20.0,0.0,0,N/A,1,0.0,N/A,N/A,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
Attacking Closeouts,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
Catch & Shoot,1.5,80.0,0,50.0,1.5,80.0,50.0,0,N/A,4,50.0,N/A,N/A,N/A,0,N/A,0,N/A


In [11]:
LoveSeriesStats = PPP(ovr_series_player_data['love'])
LoveSeriesStats.to_csv(f'Love/PPP Stats/{prevGames[-1] + Opp}.csv')
LoveSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.0,10.7,0,0.0,0.0,11.5,0.0,0,N/A,3,0.0,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.0,3.6,0,0.0,0.0,3.8,0.0,0,N/A,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,3.0,10.7,0,100.0,3.0,11.5,100.0,0,N/A,3,100.0,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,2.27,9.4,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,2.27,132.0,N/A,0,N/A,0,N/A
Attacking Closeouts,1.5,21.4,0,50.0,1.5,23.1,50.0,0,N/A,6,50.0,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,0.6,35.7,0,20.0,0.6,38.5,20.0,0,N/A,10,20.0,N/A,0.0,N/A,0,N/A,0,N/A


In [12]:
loveStats = PPP(ovr_player_data['love'])
loveStats.to_csv('!PPP Stats/loveStats.csv')
loveStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.0,4.0,0,0.0,0.0,4.8,0.0,0,N/A,2,0.0,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,0.0,4.0,1,0.0,0.0,2.4,0.0,1,0.0,0,N/A,0.0,12.5,N/A,0,N/A,0,N/A
DHO Ball Handler,0.0,4.0,1,0.0,0.0,2.4,0.0,0,N/A,1,0.0,0.0,12.5,N/A,0,N/A,0,N/A
DHO Screener,0.0,2.0,0,0.0,0.0,2.4,0.0,0,N/A,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,3.0,4.0,0,100.0,3.0,4.8,100.0,0,N/A,2,100.0,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.5,16.0,0,25.0,0.29,16.7,14.3,7,14.3,0,N/A,2.0,12.5,100.0,1,100.0,0,N/A
TRANSITION,1.78,13.5,0,60.0,1.5,9.5,50.0,0,N/A,4,50.0,2.17,34.5,100.0,1,100.0,0,N/A
Attacking Closeouts,1.5,8.0,0,50.0,1.5,9.5,50.0,0,N/A,4,50.0,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,1.33,30.0,1,50.0,1.21,33.3,46.2,1,100.0,12,41.7,3.0,12.5,100.0,0,N/A,1,100.0


In [13]:
newLowryStats = PPP(new_player_data['lowry'])
newLowryStats.to_csv(f'Lowry/PPP Stats/{prevGames[-1] + Opp}.csv')
newLowryStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.67,33.3,0,33.3,1.0,25.0,50.0,1,100.0,1,0.0,0.5,40.0,25.0,3,33.3,1,0.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,1.5,11.1,0,50.0,3.0,12.5,100.0,0,N/A,1,100.0,0.0,10.0,0.0,1,0.0,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,2.0,5.6,0,100.0,2.0,12.5,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,1.0,11.1,1,100.0,N/A,0.0,N/A,0,N/A,0,N/A,1.0,20.0,100.0,1,100.0,0,N/A
Attacking Closeouts,1.67,16.7,0,66.7,2.0,12.5,100.0,1,100.0,0,N/A,1.5,20.0,50.0,1,0.0,1,100.0
Catch & Shoot,0.0,16.7,0,0.0,0.0,25.0,0.0,0,N/A,2,0.0,0.0,10.0,0.0,1,0.0,0,N/A


In [14]:
newLowrySeriesStats = PPP(ovr_series_player_data['lowry'])
newLowrySeriesStats.to_csv(f'Lowry/PPP Stats/{prevGames[-1] + Opp}.csv')
newLowrySeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.75,44.9,0,65.2,1.82,48.4,58.3,5,20.0,7,85.7,1.67,41.2,72.7,9,77.8,2,50.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.5,15.4,0,20.0,0.83,18.2,33.3,1,100.0,5,20.0,0.0,12.5,0.0,4,0.0,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.25,12.3,6,50.0,0.5,12.1,100.0,1,100.0,0,N/A,0.0,12.5,0.0,0,N/A,1,0.0
TRANSITION,1.5,6.2,1,100.0,N/A,0.0,N/A,0,N/A,0,N/A,1.5,12.5,100.0,3,100.0,0,N/A
Attacking Closeouts,1.25,6.2,0,50.0,2.0,3.0,100.0,1,100.0,0,N/A,1.0,9.4,33.3,1,0.0,2,50.0
Catch & Shoot,1.29,10.8,0,42.9,1.5,18.2,50.0,0,N/A,6,50.0,0.0,3.1,0.0,1,0.0,0,N/A


In [15]:
lowryStats = PPP(ovr_player_data['lowry'])
lowryStats.to_csv('!PPP Stats/lowryStats.csv')
lowryStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.46,46.0,8,58.1,1.39,51.8,51.0,23,21.7,26,76.9,1.57,39.1,72.0,17,70.6,8,75.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.71,10.4,0,33.3,0.94,14.5,43.8,6,100.0,10,10.0,0.0,5.5,0.0,4,0.0,1,0.0
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.71,11.2,9,53.8,0.73,10.0,80.0,5,80.0,0,N/A,0.7,12.6,37.5,2,100.0,6,16.7
TRANSITION,1.76,13.9,2,84.0,1.29,3.5,33.3,2,0.0,1,100.0,1.83,26.4,90.9,17,94.1,5,80.0
Attacking Closeouts,0.92,6.5,1,41.7,1.5,3.6,75.0,4,75.0,0,N/A,0.67,9.9,25.0,1,0.0,7,28.6
Catch & Shoot,0.88,8.5,0,29.4,0.94,14.5,31.2,0,N/A,16,31.2,0.0,1.1,0.0,1,0.0,0,N/A


In [16]:
newMartinStats = PPP(new_player_data['martin'])
newMartinStats.to_csv(f'Martin/PPP Stats/{prevGames[-1] + Opp}.csv')
newMartinStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.0,8.3,0,0.0,0.0,11.1,0.0,0,N/A,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.0,8.3,0,0.0,0.0,11.1,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,2.0,16.7,0,100.0,2.0,22.2,100.0,2,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,1.25,33.3,0,50.0,0.0,11.1,0.0,0,N/A,1,0.0,1.67,100.0,66.7,1,100.0,2,50.0
Catch & Shoot,2.0,25.0,0,66.7,2.0,33.3,66.7,0,N/A,3,66.7,N/A,0.0,N/A,0,N/A,0,N/A


In [17]:
newMartinSeriesStats = PPP(ovr_series_player_data['martin'])
newMartinSeriesStats.to_csv(f'Martin/PPP Stats/{prevGames[-1] + Opp}.csv')
newMartinSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.0,6.7,0,0.0,0.0,10.7,0.0,2,0.0,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.0,2.2,0,0.0,0.0,3.6,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,0.0,15.6,0,0.0,0.0,10.7,0.0,3,0.0,0,N/A,0.0,23.5,0.0,1,0.0,3,0.0
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.0,17.8,3,0.0,0.0,28.6,0.0,2,0.0,3,0.0,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,2.0,4.4,0,100.0,2.0,7.1,100.0,2,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,1.17,26.7,0,41.7,0.0,7.1,0.0,1,0.0,1,0.0,1.4,58.8,50.0,1,100.0,9,44.4
Catch & Shoot,2.25,17.8,0,75.0,2.25,28.6,75.0,0,N/A,8,75.0,N/A,0.0,N/A,0,N/A,0,N/A


In [18]:
martinStats = PPP(ovr_player_data['martin'])
martinStats.to_csv('!PPP Stats/martinStats.csv')
martinStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.25,8.5,1,13.3,0.25,11.0,13.3,14,14.3,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,0.67,1.6,0,33.3,0.67,2.1,33.3,1,100.0,2,0.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.75,4.3,0,25.0,0.75,5.5,25.0,6,0.0,2,100.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,0.43,7.4,0,14.3,0.0,2.1,0.0,3,0.0,0,N/A,0.55,26.2,18.2,5,0.0,6,33.3
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.49,17.5,5,25.9,0.54,20.5,26.9,16,43.8,10,0.0,0.0,7.1,0.0,0,N/A,1,0.0
TRANSITION,1.23,9.5,3,71.4,1.21,10.2,66.7,11,72.7,1,0.0,1.33,7.1,100.0,2,100.0,0,N/A
Attacking Closeouts,1.24,20.1,0,51.4,1.23,15.0,57.1,16,62.5,5,40.0,1.25,38.1,43.8,1,100.0,15,40.0
Catch & Shoot,1.95,23.4,0,65.9,1.93,29.5,65.1,1,100.0,42,64.3,3.0,2.4,100.0,0,N/A,1,100.0


In [19]:
newRobinsonStats = PPP(new_player_data['robinson'])
newRobinsonStats.to_csv(f'Robinson/PPP Stats/{prevGames[-1] + Opp}.csv')
newRobinsonStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,1.8,64.7,0,66.7,3.0,25.0,100.0,0,N/A,1,100.0,1.39,144.0,50.0,1,0.0,1,100.0
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,2.0,50.0,0,66.7,2.0,75.0,66.7,0,N/A,3,66.7,N/A,0.0,N/A,0,N/A,0,N/A


In [20]:
newRobinsonSeriesStats = PPP(ovr_series_player_data['robinson'])
newRobinsonSeriesStats.to_csv(f'Robinson/PPP Stats/{prevGames[-1] + Opp}.csv')
newRobinsonSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.0,2.8,1,N/A,0.0,3.3,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,1.19,16.3,0,40.0,1.5,6.7,50.0,0,N/A,2,50.0,1.03,64.7,33.3,1,0.0,2,50.0
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,3.0,8.3,0,100.0,3.0,10.0,100.0,0,N/A,3,100.0,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,0.0,2.8,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,16.7,0.0,1,0.0,0,N/A
Attacking Closeouts,1.84,28.7,0,88.9,1.8,27.7,85.7,3,100.0,4,75.0,2.0,33.3,100.0,2,100.0,0,N/A
Catch & Shoot,1.5,16.7,0,50.0,1.5,20.0,50.0,0,N/A,6,50.0,N/A,0.0,N/A,0,N/A,0,N/A


In [21]:
robinsonStats = PPP(ovr_player_data['robinson'])
robinsonStats.to_csv('!PPP Stats/robinsonStats.csv')
robinsonStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.12,11.1,5,63.6,0.67,11.3,42.9,2,50.0,5,40.0,2.5,10.5,100.0,2,100.0,2,100.0
PNR Screener,0.0,0.7,0,0.0,0.0,0.9,0.0,0,N/A,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.79,22.1,2,32.1,0.84,17.9,31.6,5,40.0,14,28.6,0.71,33.6,33.3,2,50.0,7,28.6
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.22,6.2,1,50.0,1.22,8.5,50.0,3,33.3,5,60.0,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,0.4,6.9,1,22.2,0.67,2.8,33.3,1,100.0,2,0.0,0.29,18.4,16.7,6,16.7,0,N/A
Attacking Closeouts,1.48,19.7,1,73.1,1.35,12.6,58.3,3,100.0,9,44.4,1.6,39.5,85.7,13,92.3,1,0.0
Catch & Shoot,1.38,16.7,1,47.8,1.38,22.6,47.8,0,N/A,23,47.8,N/A,0.0,N/A,0,N/A,0,N/A


In [22]:
newStrusStats = PPP(new_player_data['strus'])
newStrusStats.to_csv(f'Strus/PPP Stats/{prevGames[-1] + Opp}.csv')
newStrusStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.43,34.9,0,50.0,0.0,28.6,0.0,2,0.0,0,N/A,2.43,41.1,100.0,1,100.0,1,100.0
PNR Screener,0.0,7.1,0,0.0,0.0,14.3,0.0,0,N/A,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,2.11,23.7,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,2.11,47.4,100.0,1,100.0,1,100.0
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,2.0,7.1,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,2.0,14.3,100.0,1,100.0,0,N/A
Attacking Closeouts,0.0,7.1,0,0.0,0.0,14.3,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,1.5,14.3,0,50.0,1.5,28.6,50.0,0,N/A,2,50.0,N/A,0.0,N/A,0,N/A,0,N/A


In [23]:
newStrusSeriesStats = PPP(ovr_series_player_data['strus'])
newStrusSeriesStats.to_csv(f'Strus/PPP Stats/{prevGames[-1] + Opp}.csv')
newStrusSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.19,8.2,0,40.0,0.0,6.0,0.0,3,0.0,0,N/A,2.43,13.1,100.0,1,100.0,1,100.0
PNR Screener,2.57,9.7,0,85.7,2.57,14.0,85.7,0,N/A,7,85.7,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,1.1,24.1,0,50.0,0.0,6.0,0.0,0,N/A,3,0.0,1.33,65.1,61.5,9,77.8,4,25.0
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,2.0,1.4,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,2.0,4.5,100.0,1,100.0,0,N/A
Attacking Closeouts,1.41,13.8,0,66.7,1.01,11.9,0.0,1,0.0,1,0.0,2.0,18.2,100.0,4,100.0,0,N/A
Catch & Shoot,0.44,37.5,0,14.8,0.44,54.0,14.8,0,N/A,27,14.8,N/A,0.0,N/A,0,N/A,0,N/A


In [24]:
strusStats = PPP(ovr_player_data['strus'])
strusStats.to_csv('!PPP Stats/strusStats.csv')
strusStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.0,14.1,2,41.7,0.8,13.7,33.3,10,20.0,8,50.0,1.6,15.6,66.7,4,75.0,2,50.0
PNR Screener,2.57,3.7,0,85.7,2.57,4.8,85.7,0,N/A,7,85.7,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.73,19.6,0,28.6,0.55,7.5,18.2,0,N/A,11,18.2,0.8,59.5,33.3,19,36.8,5,20.0
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.0,1.6,0,0.0,0.0,2.1,0.0,1,0.0,2,0.0,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,2.05,5.1,0,100.0,2.06,5.3,100.0,5,100.0,1,100.0,2.0,4.5,100.0,2,100.0,0,N/A
Attacking Closeouts,1.23,12.0,1,58.8,0.81,10.2,22.2,4,50.0,5,0.0,2.0,18.2,100.0,8,100.0,0,N/A
Catch & Shoot,0.46,35.7,0,14.9,0.45,45.9,14.9,0,N/A,67,14.9,1.14,2.0,N/A,0,N/A,0,N/A


In [25]:
newVincentStats = PPP(new_player_data['vincent'])
newVincentStats.to_csv(f'Vincent/PPP Stats/{prevGames[-1] + Opp}.csv')
newVincentStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.0,42.9,0,0.0,0.0,27.3,0.0,2,0.0,1,0.0,0.0,100.0,0.0,2,0.0,1,0.0
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.0,7.1,0,0.0,0.0,9.1,0.0,0,N/A,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,2.27,6.3,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,2.27,29.3,N/A,0,N/A,0,N/A
Attacking Closeouts,0.6,23.7,0,0.0,0.6,30.2,0.0,2,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,1.0,21.4,0,33.3,1.0,27.3,33.3,0,N/A,3,33.3,N/A,0.0,N/A,0,N/A,0,N/A


In [26]:
newVincentSeriesStats = PPP(ovr_series_player_data['vincent'])
newVincentSeriesStats.to_csv(f'Vincent/PPP Stats/{prevGames[-1] + Opp}.csv')
newVincentSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.14,43.2,3,57.1,0.92,36.8,50.0,16,62.5,6,16.7,1.54,62.3,69.2,7,71.4,6,66.7
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.9,7.3,0,0.0,1.29,6.8,0.0,0,N/A,2,0.0,0.0,8.7,0.0,2,0.0,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,2.2,5.5,0,80.0,N/A,0.0,N/A,0,N/A,0,N/A,2.2,21.7,80.0,1,100.0,4,75.0
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.0,3.3,0,0.0,0.0,4.4,0.0,0,N/A,3,0.0,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,2.12,5.7,0,100.0,2.08,6.4,100.0,3,100.0,0,N/A,2.27,3.8,N/A,0,N/A,0,N/A
Attacking Closeouts,0.63,6.9,0,20.0,0.75,7.8,25.0,4,25.0,0,N/A,0.0,4.3,0.0,0,N/A,1,0.0
Catch & Shoot,2.04,27.5,0,68.0,2.04,36.8,68.0,0,N/A,25,68.0,N/A,0.0,N/A,0,N/A,0,N/A


In [27]:
vincentStats = PPP(ovr_player_data['vincent'])
vincentStats.to_csv('!PPP Stats/vincentStats.csv')
vincentStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.28,40.2,6,59.0,1.15,37.1,53.0,42,61.9,24,37.5,1.53,47.8,70.6,18,83.3,16,56.2
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,0.51,9.5,1,9.5,0.76,7.6,10.0,1,0.0,9,11.1,0.18,14.1,9.1,11,9.1,0,N/A
DHO Screener,3.0,0.4,0,100.0,N/A,0.0,N/A,0,N/A,0,N/A,3.0,1.3,100.0,0,N/A,1,100.0
DBL Ball Handler,1.42,6.3,0,56.2,1.0,1.6,33.3,2,0.0,1,100.0,1.51,17.8,61.5,5,100.0,8,37.5
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.34,5.5,3,10.0,0.39,6.6,10.0,5,20.0,5,0.0,0.0,2.6,N/A,0,N/A,0,N/A
TRANSITION,1.9,4.1,0,75.0,2.26,2.8,100.0,3,100.0,1,100.0,1.56,7.4,50.0,1,100.0,3,33.3
Attacking Closeouts,0.72,9.3,0,27.3,0.84,9.9,31.2,15,33.3,1,0.0,0.33,7.7,16.7,1,100.0,5,0.0
Catch & Shoot,1.86,23.3,0,61.9,1.86,32.8,61.9,0,N/A,63,61.9,N/A,0.0,N/A,0,N/A,0,N/A


In [28]:
newZellerStats = PPP(new_player_data['zeller'])
newZellerStats.to_csv(f'Zeller/PPP Stats/{prevGames[-1] + Opp}.csv')
newZellerStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
PNR Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
Attacking Closeouts,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A
Catch & Shoot,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,N/A,N/A,0,N/A,0,N/A


In [29]:
newZellerSeriesStats = PPP(ovr_series_player_data['zeller'])
newZellerSeriesStats.to_csv(f'Zeller/PPP Stats/{prevGames[-1] + Opp}.csv')
newZellerSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,2.0,25.0,0,100.0,2.0,27.3,100.0,3,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,0.0,33.3,3,0.0,0.0,36.4,0.0,1,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [30]:
zellerStats = PPP(ovr_player_data['zeller'])
zellerStats.to_csv('!PPP Stats/zellerStats.csv')
zellerStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
PNR Screener,1.25,22.2,1,71.4,1.6,19.2,80.0,5,80.0,0,N/A,0.67,30.0,50.0,1,100.0,1,0.0
DHO Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DHO Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,2.0,2.8,0,100.0,2.0,3.8,100.0,1,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
TRANSITION,0.0,2.4,0,N/A,0.0,3.4,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Attacking Closeouts,0.0,25.0,3,0.0,0.0,34.6,0.0,6,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
Catch & Shoot,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A


In [33]:
newTeamStats = PPP(new_player_data['team'])
newTeamStats.to_csv(f'Team/PPP Stats/{prevGames[-1] + Opp}.csv')
newTeamStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.69,22.9,1,30.0,0.53,17.9,23.5,12,25.0,5,20.0,0.87,33.8,38.5,9,44.4,4,25.0
PNR Screener,0.76,5.7,0,28.6,0.76,8.3,28.6,6,33.3,1,0.0,N/A,0.0,N/A,0,N/A,0,N/A
DHO Ball Handler,1.52,8.1,0,55.6,1.5,4.2,50.0,0,N/A,4,50.0,1.53,16.4,60.0,3,33.3,2,100.0
DHO Screener,1.31,4.9,0,50.0,1.04,3.0,50.0,2,50.0,0,N/A,1.5,9.1,50.0,0,N/A,4,50.0
DBL Ball Handler,2.0,1.4,0,100.0,2.0,2.1,100.0,2,100.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.91,14.2,0,38.9,1.02,13.4,45.5,10,50.0,1,0.0,0.71,15.9,28.6,3,33.3,4,25.0
TRANSITION,1.48,8.3,1,71.4,1.66,7.0,75.0,4,75.0,0,N/A,1.23,11.1,66.7,3,66.7,0,N/A
Attacking Closeouts,0.92,10.9,1,33.3,0.59,10.7,14.3,6,16.7,1,0.0,1.6,11.4,60.0,2,50.0,3,66.7
Catch & Shoot,1.26,13.7,0,42.1,1.33,18.9,44.4,0,N/A,18,44.4,0.0,2.3,0.0,1,0.0,0,N/A


In [34]:
newOppStats = PPP(new_player_data['opp'])
newOppStats.to_csv(f'Opp/PPP Stats/{prevGames[-1] + Opp}.csv')
newOppStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.92,21.0,6,56.5,1.08,13.5,58.3,12,58.3,0,N/A,0.79,37.4,54.5,8,75.0,3,0.0
PNR Screener,1.33,4.3,0,66.7,2.0,3.1,100.0,3,100.0,0,N/A,0.67,6.8,33.3,2,50.0,1,0.0
DHO Ball Handler,1.7,4.2,1,100.0,1.64,5.1,100.0,2,100.0,1,100.0,2.0,2.3,100.0,1,100.0,0,N/A
DHO Screener,2.25,3.5,0,100.0,2.0,1.0,100.0,1,100.0,0,N/A,2.32,8.8,100.0,2,100.0,1,100.0
DBL Ball Handler,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
DBL Screener,N/A,0.0,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,0.92,16.3,2,47.4,0.56,18.5,28.6,13,30.8,1,0.0,2.2,11.4,100.0,4,100.0,1,100.0
TRANSITION,0.82,13.1,2,33.3,1.04,12.0,50.0,4,75.0,2,0.0,0.44,15.6,16.7,0,N/A,6,16.7
Attacking Closeouts,1.69,6.3,0,75.0,1.8,5.2,80.0,4,75.0,1,100.0,1.55,8.8,66.7,2,100.0,1,0.0
Catch & Shoot,0.69,9.3,1,25.0,0.75,12.5,25.0,0,N/A,12,25.0,0.0,2.3,N/A,0,N/A,0,N/A


In [35]:
TeamSeriesStats = PPP(ovr_series_player_data['team'])
TeamSeriesStats.to_csv(f'Team/PPP Stats/{prevGames[-1] + Opp}.csv')
TeamSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.09,22.6,11,45.2,0.71,19.6,30.1,81,29.6,32,31.2,1.68,29.6,68.5,41,63.4,32,75.0
PNR Screener,1.48,6.5,6,66.7,1.53,7.8,63.4,31,58.1,10,80.0,1.23,3.4,100.0,4,100.0,0,N/A
DHO Ball Handler,0.92,6.9,0,33.3,0.77,4.4,20.0,6,16.7,19,21.1,1.04,12.7,43.8,22,45.5,10,40.0
DHO Screener,0.93,5.3,4,40.0,1.01,3.3,50.0,20,50.0,0,N/A,0.87,9.6,30.0,3,66.7,17,23.5
DBL Ball Handler,2.0,1.2,0,81.8,2.0,0.6,100.0,4,100.0,0,N/A,2.0,2.4,71.4,1,100.0,6,66.7
DBL Screener,2.8,0.5,0,100.0,2.8,0.8,100.0,1,100.0,4,100.0,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.01,16.6,13,46.2,1.0,16.7,45.6,71,50.7,19,26.3,1.02,16.4,47.6,16,75.0,26,30.8
TRANSITION,1.49,5.4,3,65.6,1.54,5.0,66.7,21,66.7,0,N/A,1.42,6.4,63.6,10,70.0,1,0.0
Attacking Closeouts,1.17,10.1,7,50.0,1.03,10.2,43.5,25,48.0,21,38.1,1.5,9.8,60.7,11,81.8,17,47.1
Catch & Shoot,1.4,13.9,0,46.9,1.42,19.7,47.7,1,100.0,127,47.2,0.0,0.7,0.0,2,0.0,0,N/A


In [36]:
OppSeriesStats = PPP(ovr_series_player_data['opp'])
OppSeriesStats.to_csv(f'Opp/PPP Stats/{prevGames[-1] + Opp}.csv')
OppSeriesStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,0.98,16.0,17,54.5,1.02,11.8,56.5,40,57.5,6,50.0,0.95,24.8,52.4,22,77.3,20,25.0
PNR Screener,1.42,3.2,4,82.4,1.51,3.0,90.0,8,100.0,2,50.0,1.25,3.6,71.4,6,83.3,1,0.0
DHO Ball Handler,1.02,6.2,8,46.4,0.51,5.5,29.4,11,36.4,6,16.7,1.78,7.6,72.7,5,100.0,6,50.0
DHO Screener,1.19,5.2,0,41.4,1.89,3.5,72.7,5,100.0,6,50.0,0.62,8.7,22.2,11,27.3,7,14.3
DBL Ball Handler,0.0,0.4,0,0.0,N/A,0.0,N/A,0,N/A,0,N/A,0.0,1.4,0.0,0,N/A,3,0.0
DBL Screener,0.5,0.6,0,25.0,0.0,0.7,0.0,0,N/A,3,0.0,2.0,0.5,100.0,1,100.0,0,N/A
ISOLATION,1.1,21.1,15,51.4,0.93,22.2,46.3,70,54.3,12,0.0,1.51,19.1,65.5,12,100.0,17,41.2
TRANSITION,1.47,13.4,3,62.0,1.3,10.6,60.6,30,66.7,3,0.0,1.65,19.1,63.2,14,71.4,24,58.3
Attacking Closeouts,1.49,6.5,0,61.5,1.16,6.2,48.0,17,58.8,8,25.0,2.09,7.1,85.7,6,100.0,8,75.0
Catch & Shoot,1.46,11.8,1,48.0,1.48,17.3,48.0,0,N/A,75,48.0,0.0,0.5,N/A,0,N/A,0,N/A


In [37]:
teamStats = PPP(ovr_player_data['team'])
teamStats.to_csv('!PPP Stats/teamStats.csv')
teamStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.06,21.1,36,44.8,0.85,19.9,35.9,216,31.9,96,44.8,1.47,23.9,63.3,81,64.2,69,62.3
PNR Screener,1.22,5.3,27,64.4,1.25,6.2,62.5,66,62.1,14,64.3,1.06,2.9,80.0,5,100.0,5,60.0
DHO Ball Handler,0.69,6.6,4,25.8,0.77,5.0,26.8,25,36.0,57,22.8,0.6,10.4,24.6,46,26.1,23,21.7
DHO Screener,0.83,5.2,6,34.5,0.55,2.7,25.6,42,26.2,1,0.0,1.0,11.0,39.7,26,57.7,47,29.8
DBL Ball Handler,1.2,1.2,0,50.0,1.38,0.5,62.5,6,66.7,2,50.0,1.14,3.1,45.5,6,100.0,16,25.0
DBL Screener,2.4,0.4,0,100.0,2.4,0.6,100.0,6,100.0,4,100.0,N/A,0.0,N/A,0,N/A,0,N/A
ISOLATION,1.02,20.4,41,47.2,0.93,20.7,43.1,262,46.2,49,26.5,1.23,19.9,56.9,51,80.4,79,41.8
TRANSITION,1.46,7.7,16,71.7,1.45,6.4,73.3,76,76.3,10,50.0,1.46,10.7,69.7,46,80.4,20,45.0
Attacking Closeouts,1.03,8.7,16,47.3,0.91,8.5,41.5,76,50.0,42,26.2,1.28,9.1,57.6,30,90.0,36,30.6
Catch & Shoot,1.39,13.0,2,47.5,1.39,18.3,47.5,9,100.0,311,46.0,1.43,0.7,50.0,2,0.0,2,100.0


In [38]:
oppStats = PPP(ovr_player_data['opp'])
oppStats.to_csv('!PPP Stats/oppStats.csv')
oppStats

,Total PPP,% of Poss.,Total TO,Total FG%,Shooting PPP,% of Shooting Poss.,Shooting FG%,Shooting 2pt Att.,Shooting 2pt FG%,Shooting 3pt Att.,Shooting 3pt FG%,Passing PPP,% of Passing Poss.,Passing FG%,Passing 2pt Att.,Passing 2pt FG%,Passing 3pt Att.,Passing 3pt FG%
PNR Ball Handler,1.0,15.3,42,50.2,0.9,12.2,46.4,114,52.6,37,27.0,1.13,21.8,55.1,53,79.2,65,35.4
PNR Screener,1.39,4.5,14,70.3,1.53,4.5,73.6,34,88.2,19,47.4,1.11,4.5,61.9,10,80.0,11,45.5
DHO Ball Handler,1.0,4.0,11,43.1,0.76,3.5,35.7,22,50.0,20,20.0,1.36,5.1,56.5,13,69.2,10,40.0
DHO Screener,0.98,3.4,1,35.5,1.59,1.8,57.9,12,66.7,7,42.9,0.64,6.9,25.6,24,37.5,19,10.5
DBL Ball Handler,0.92,0.6,0,41.7,0.67,0.2,33.3,3,33.3,0,N/A,1.0,1.3,44.4,3,100.0,6,16.7
DBL Screener,1.55,0.5,0,72.7,1.29,0.5,57.1,3,100.0,4,25.0,2.0,0.6,100.0,4,100.0,0,N/A
ISOLATION,1.02,21.6,48,48.0,0.92,21.4,45.3,207,52.7,49,14.3,1.24,22.0,54.0,42,83.3,71,36.6
TRANSITION,1.22,13.6,24,54.1,1.15,11.4,53.3,98,61.2,22,18.2,1.31,18.5,54.9,44,75.0,58,39.7
Attacking Closeouts,1.13,9.3,12,49.1,1.02,8.9,45.5,72,52.8,29,27.6,1.35,9.9,55.0,19,89.5,41,39.0
Catch & Shoot,1.11,14.2,4,37.4,1.13,20.4,37.5,2,100.0,289,37.0,0.4,0.7,33.3,1,100.0,2,0.0
